# Advanced SQL

## Functions and Procedures

We have already seen several functions that are built into the SQL language. In this section, we show how developers can write their ow functions and procedures, and **store them in the database** so as to invoke them from SQL statements. 

SQL allows for the definition of functions, procedures, and methods.

### Declaring and Invoking MSSQL functions and procedures

Suppose that we want a function that, given the name of a department, returns the count of the number of instructors in that department. 

In [6]:
-- select the database
USE uni;
GO
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[instructor_count]'))
    BEGIN
        DROP FUNCTION dbo.instructor_count
    END;
GO

-- define the function
CREATE FUNCTION dbo.instructor_count(@dept_name VARCHAR(20))
RETURNS INT 
AS
BEGIN
    -- Declare the return variable here
    DECLARE @instructor_n INT
    -- Make the query
    SELECT @instructor_n = COUNT(*)
    FROM instructor
    WHERE instructor.dept_name = @dept_name
    -- return
    RETURN @instructor_n
END;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.017

In [4]:
-- test it
SELECT dbo.instructor_count('Statistics') AS inst_n;
GO

SELECT COUNT(*) AS inst_n
FROM instructor
WHERE instructor.dept_name = 'Statistics';
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.036

inst_n
6


inst_n
6


This function returns all the professors names as well as ids for a department. This is an example of a table function which is relatively different in its definition from scalar output functions such as the one above.

In [7]:
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[dept_instructors]'))
    BEGIN
        DROP FUNCTION dbo.dept_instructors
    END;
GO

CREATE FUNCTION dbo.dept_instructors(@dept_name VARCHAR(20))
RETURNS TABLE
AS 
RETURN(
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = @dept_name 
);
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.012

In [8]:
-- test it
SELECT * FROM dbo.dept_instructors('Statistics');
GO

SELECT [name], ID 
FROM instructor 
WHERE dept_name = 'Statistics';
GO

(6 rows affected)

(6 rows affected)

Total execution time: 00:00:00.018

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


Below we generate a stored procedure which does exactly the same.

In [9]:
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[dept_instructors_proc]'))
    BEGIN
        DROP PROCEDURE dbo.dept_instructors_proc
    END;
GO

-- define the stored procedure
CREATE PROCEDURE dbo.dept_instructors_proc(@dept_name VARCHAR(20))
AS
BEGIN
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = @dept_name
END;
GO

-- Running it
EXEC dept_instructors_proc @dept_name = 'Statistics';
GO

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.018

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


### Stored procedures versus functions

While the output is the same, there are differences beyound just how they are coded:

- **Functions are computed values and cannot perform permanent changes to the server, whereas stored procedures can**
- **[The output of] functions can be using within SQL query, stored procedures, since they need the ``EXEC`` statement to be ran, cannot.**
- **Stored procedures can return zero, single or multiple values; functions must return a single value, being either a scalar or a table**
- **We can use exception handling using Try-Catch blocks in stored procedures, but we cannot within functions**
- **We can define input paramters for functions; we can define both input as well as output parameters for stored procedures**

Found this [thread](https://stackoverflow.com/questions/1179758/function-vs-stored-procedure-in-sql-server) really helpful.

In [16]:
-- a stored procedure with an output parameter
-- existing, drop it
IF EXISTS (SELECT  * FROM dbo.sysobjects WHERE id = OBJECT_ID(N'[dbo].[dept_instructors_proc2]'))
    BEGIN
        DROP PROCEDURE dbo.dept_instructors_proc2
    END;
GO

-- define the stored procedure
CREATE PROCEDURE dbo.dept_instructors_proc2 (
    @dept_name VARCHAR(20),
    @instructor_count INT OUTPUT  
) AS
BEGIN
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = @dept_name;

    -- prepare the output
    SELECT @instructor_count = @@ROWCOUNT;
END;
GO

-- declare the output parameter
DECLARE @count INT
-- Running it
EXEC dept_instructors_proc2 @dept_name = 'Statistics', @instructor_count = @count OUTPUT
-- retrieve the output
SELECT @count AS 'Number of instructors found';
GO

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

(1 row affected)

Total execution time: 00:00:00.014

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


Number of instructors found
6


## Control-of-Flow statements

One of the nice things of functions and stored procedures is that we can make use of several control of flow statements.

### The `BEGIN...END` statement

The BEGIN...END statement is used to define a statement block. A statement block consists of a set of SQL statements that execute together. A statement block is also known as a batch.

```
BEGIN
    { sql_statement | statement_block}
END
```

For example, in the code belo we combine a sql query and printing a message conditional on the number of rows in the output table of that query. We can do this because we included them in a batch. Note that ``@@ROWCOUNT`` is a system variable that returns the number of rows affected by the last previous statement.

In [17]:
BEGIN
    SELECT [name], ID 
    FROM instructor 
    WHERE dept_name = 'Statistics';

IF @@ROWCOUNT > 5
    PRINT 'There are more than 5 instructors in the department'
ELSE
    PRINT 'There are less than 5 instructors in the department'
END;
GO

(6 rows affected)

There are more than 5 instructors in the department

Total execution time: 00:00:00.003

name,ID
Atanassov,28400
Arias,37687
Gutierrez,64871
Pingr,78699
Choll,90643
Arinb,95030


We can also nest several begin end statements. For example

In [23]:
BEGIN 
    -- declare the variables we will use in this batch
    DECLARE @student_name VARCHAR(40)
    DECLARE @dept_name VARCHAR(40)
    DECLARE @tc INT

    -- the query: highest total credits
    SELECT TOP 1
        @student_name = [name],
        @dept_name = dept_name,
        @tc = tot_cred
    FROM student
    ORDER BY tot_cred DESC;

    -- second level
    IF @dept_name IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'The student with the highest total credits is ' + @student_name  +' (STEM FIELD) with ' + CONVERT(varchar(10), @tc)
        END
    ELSE
        BEGIN
            PRINT 'The student with the highest total credits is ' + @student_name + ' (NON-STEM FIELD) with ' + CONVERT(varchar(10), @tc)
        END;
END;

The student with the highest total credits is Yoneda (NON-STEM FIELD) with 129

Total execution time: 00:00:00.004

### ``IF...ELSE``

When the condition in the IF clause evaluates to FALSE and you want to execute another statement block, you can use the ELSE clause.

```
IF Boolean_expression
BEGIN
    -- Statement block executes when the Boolean expression is TRUE
END
ELSE
BEGIN
    -- Statement block executes when the Boolean expression is FALSE
END
```
For example, say that we want to check whether more than half of the students in a certain class failed

In [39]:
BEGIN
    -- Declare the variables to be used in the batch
    DECLARE @rel_dept AS VARCHAR(20)
    SET @rel_dept = 'Statistics'
    DECLARE @percent_fail AS FLOAT
    -- the query
    SET @percent_fail = (
        SELECT AVG(CAST(IIF(takes.grade LIKE 'C_', 1, 0) AS FLOAT))
        FROM takes, student, course
        WHERE 
            takes.ID = student.ID AND 
            course.course_id = takes.course_id AND 
            course.dept_name = @rel_dept
    );
    -- messages
    IF (@percent_fail > 0.5)
        BEGIN
            PRINT 'More than half of students failed, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE
        BEGIN
            PRINT 'Less than half of students failed, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END;
END;


Less than half of students failed, namely 35.31%

Total execution time: 00:00:00.057

Note that we can nest them too and use ``ELSE IF``.

In [47]:
BEGIN
    -- Declare the variables to be used in the batch
    DECLARE @rel_dept AS VARCHAR(20)
    SET @rel_dept = 'Biology'
    DECLARE @percent_fail AS FLOAT
    -- the query
    SET @percent_fail = (
        SELECT AVG(CAST(IIF(takes.grade LIKE 'C_', 1, 0) AS FLOAT))
        FROM takes, student, course
        WHERE 
            takes.ID = student.ID AND 
            course.course_id = takes.course_id AND 
            course.dept_name = @rel_dept
    );
    -- messages
    IF (@percent_fail > 0.5) AND @rel_dept IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'More than half of students failed in this STEM-field, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE IF (@percent_fail > 0.5) AND @rel_dept IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'More than half of students failed in this STEM-field, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE IF (@percent_fail < 0.5) AND @rel_dept IN ('Biology', 'Comp. Sci.', 'Statistics', 'Math', 'Mech Eng.', 'Civil Eng.')
        BEGIN
            PRINT 'More than half of students failed in this STEM-field, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END
    ELSE
        BEGIN
            PRINT 'Less than half of students failed, namely ' + CONVERT(VARCHAR(10), ROUND(@percent_fail * 100, 2)) + '%'
        END;
END;

More than half of students failed in this STEM-field, namely 33.96%

Total execution time: 00:00:00.032

### `WHILE`

The WHILE statement is a control-flow statement that allows you to execute a statement block repeatedly as long as a specified condition is TRUE.

First, the Boolean_expression is an expression that evaluates to TRUE or FALSE.

Second, <code>sql_statement | statement_block</code> is any Transact-SQL statement or a set of Transact-SQL statements. A statement block is defined using the <code>BEGIN...END</code> statement.

If the Boolean_expression evaluates to FALSE in an iteration, no statement inside the WHILE loop will be executed.

Inside the WHILE loop, you must change some variables to make the Boolean_expression returns FALSE at some points. Otherwise, you will have an indefinite loop.

Furthermore:

- `BREAK` - Causes an exit from the innermost WHILE loop. Any statements that appear after the END keyword, marking the end of the loop, are executed
- `CONTINUE` - Causes the WHILE loop to restart, ignoring any statements after the `CONTINUE` keyword, i.e. skiping the current itearation

```
WHILE Boolean_expression
{ sql_statement | statement_block | BREAK | CONTINUE }

```

Below an example.Double the budget of departments with below average budgets until the average budget is higher than 3,000,000. However, never allow one of them to be above 3,000,000 after the doubling.

In [45]:
-- before
SELECT AVG(budget) FROM department;
GO

-- Start the while loop
WHILE (SELECT AVG(budget) FROM department) < 3000000
BEGIN
    -- declare the relevant variables
    DECLARE @cur_depts AS VARCHAR(20)
    -- Below average departments
    SELECT @cur_depts = dept_name
    FROM department
    WHERE budget < (SELECT AVG(budget) FROM department);
    -- alter the database
    UPDATE department
    SET budget = budget * 2;
    -- message
    PRINT 'Doubling the budget from ' + COALESCE(@cur_depts, ', ', '')
    -- no budget can exceed 3,000,000
    IF (SELECT MAX(budget) FROM department) > 3000000
        BREAK
    ELSE
        CONTINUE
END;
GO

-- after
SELECT AVG(budget) FROM department;
GO


(1 row affected)

(20 rows affected)

Doubling the budget from Statistics

(1 row affected)

Total execution time: 00:00:00.013

(No column name)
2265264.020000


(No column name)
4530528.040000


#